In [1]:
# 1 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 2 定义超参数
BATCH_SIZE = 16   # 根据电脑性能
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 优先选择gpu
EPOCHS = 10  # 训练数据集的轮次

In [3]:
# 3 构建pipeline，处理图像
pipeline = transforms.Compose([
    transforms.ToTensor(), # 将图片转化为张量形式
    transforms.Normalize((0.1307,), (0.3081,))  # 先不要纠结这两个数为什么设置成这样，先按照官网的设置就对了  分别是：标准差  均值
                                                # 正则化的作用就是，当出现过拟合现象的时候可以降低模型复杂度
    
])

In [4]:
# 4 下载，加载数据
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST('../data', train=True, transform=pipeline, download=True)   # 将transform 设置为pipeline，其实意义就是，pipeline中规定了流程和正则化要求

test_set = datasets.MNIST('../data', train=False, transform=pipeline, download=True)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)   # shuffle 就是打乱顺序，随机
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 插入代码，显示mnist中的图片
with open('../data/MNIST/raw/train-images-idx3-ubyte', 'rb') as f:
    file = f.read()

In [6]:
image1 = [int(str(item).encode('ascii'), 16) for item in file[16:16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [7]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)   # 把这个image1转化为一个数学形式，采用np.array（）的方法

print(image1_np.shape)

(28, 28, 1)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14784\1130399795.py:4: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 294 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)   # 把这个image1转化为一个数学形式，采用np.array（）的方法
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14784\1130399795.py:4: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 310 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)   # 把这个image1转化为一个数学形式，采用np.array（）的方法
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14784\1130399795.py:4: Dep

In [8]:
cv2.imwrite('image1.jpg', image1_np)   # 显示

True

In [9]:
# 5 构建网络模型
class FirstNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)   # 灰度图片的通道：1  输出通道：10  卷积核：5（5x5）
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.fc1 = nn.Linear(20*10*10, 500)   # 20*10*10 输入通道数  500 输出通道数
        self.fc2 = nn.Linear(500, 10)   # 只有是个数字，返回的是十个概率
        
    def forward(self, x):   # 前向传播
        input_size = x.size(0)   # batch_size
        x = self.conv1(x)   # 输入：batch_size*1*28*28, 输出是：batch*10*24*24  (28 - 5 + 1 = 24)  10是通道数
        x = F.relu(x)   # shape是不会变化的
        x = F.max_pool2d(x, 2, 2)   # 输入：batch*10*24*24 输出：batch*10*12*12
        
        x = self.conv2(x)   # 输入batch*10*12*12, 输出是：batch*20*10*10  (12 - 3 + 1 = 10)
        x = F.relu(x)   # shape是不会变化的
        
        x = x.view(input_size, -1)   # 拉平，拉成一排  -1 自动计算维度，2000
        
        x = self.fc1(x)   # 输入：batch*2000 输出：batch*500
        x = F.relu(x)
        
        x = self.fc2(x)   # 输入：batch*500 输出：batch*10
        
        output = F.log_softmax(x, dim=1)   # 计算分类后每个数字的概率值
        return output

In [10]:
# 6 定义优化器
model = FirstNet().to(DEVICE)

optimizer = optim.Adam(model.parameters())


In [11]:
# 7 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # training
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):      # data是数据，target是标签
        # 部署到device上
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 预测
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)   # 这个交叉熵差值函数 适合多分类
        # 反向传播
        loss.backward()
        # 优化参数
        optimizer.step()
        
        if batch_idx % 3000 == 0:
            print("Train Epoch: {} \t Loss: {:.6f}".format(epoch, loss.item()))

In [12]:
# 8 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0
    # 测试损失
    test_loss = 0
    with torch.no_grad():  # 既不计算梯度， 也不会进行反向传播
        for data, target in test_loader: 
            data, target = data.to(device), target.to(device)
        # 测试数据
            output = model(data)
        # 计算测试损失
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
        # 找到概率值最大的下标(多分类)
            pred = output.max(1, keepdim=True)[1]   # 事实上，max返回的是一个容器，里面装着0：值 1： 索引
        # 累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)
    print("Test 平均损失 Loss: {:.4f}, 准确率：{:.3f}\n".format(test_loss, 100.0 * correct / len(test_loader.dataset)))

In [13]:
# 9 调用 方法
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, test_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch: 1 	 Loss: 2.295544
Test 平均损失 Loss: 0.0926, 准确率：96.980

Train Epoch: 2 	 Loss: 0.122402
Test 平均损失 Loss: 0.0730, 准确率：97.350

Train Epoch: 3 	 Loss: 0.225913
Test 平均损失 Loss: 0.0237, 准确率：99.270

Train Epoch: 4 	 Loss: 0.005303
Test 平均损失 Loss: 0.0231, 准确率：99.220

Train Epoch: 5 	 Loss: 0.000227
Test 平均损失 Loss: 0.0203, 准确率：99.250

Train Epoch: 6 	 Loss: 0.001088
Test 平均损失 Loss: 0.0051, 准确率：99.850

Train Epoch: 7 	 Loss: 0.000013
Test 平均损失 Loss: 0.0069, 准确率：99.780

Train Epoch: 8 	 Loss: 0.000865
Test 平均损失 Loss: 0.0127, 准确率：99.530

Train Epoch: 9 	 Loss: 0.005420
Test 平均损失 Loss: 0.0040, 准确率：99.850

Train Epoch: 10 	 Loss: 0.001878
Test 平均损失 Loss: 0.0155, 准确率：99.530

